In [ ]:
import json
import pprint
import time

import requests

In [ ]:
# Get all regions' names.
query = '''{
    Region {
        name @output(out_name: "region_name")
    }
}'''

args = {}

In [ ]:
# Get all the parent regions of the "Great Sept of Baelor" region.
query = '''{
    Region @filter(op_name: "name_or_alias", value: ["$region"]) {
        out_Has_Parent_Region {
            out_Has_Parent_Region @recurse(depth: 10) {
                name @output(out_name: "parent_region")
            }
        }
    }
}'''

args = {
    'region': 'Great Sept of Baelor'
}

In [ ]:
# Get all the people that live in The Riverlands.
query = '''{
    Region @filter(op_name: "name_or_alias", value: ["$region"]) {        
        in_Has_Parent_Region @recurse(depth: 10) {
            name @output(out_name: "region_name")
            
            in_Lives_In {
                name @output(out_name: "character_name")
                
                out_Owes_Allegiance_To @fold {
                    ... on NobleHouse {
                        name @output(out_name: "loyal_to_noble_houses")
                    }
                }
            }
        }
    }
}'''

args = {
    'region': 'The Riverlands',
}

In [ ]:
# Get everyone that Sansa Stark owes allegiance to.
query = '''{
    Character @filter(op_name: "name_or_alias", value: ["$character"]) {
        out_Owes_Allegiance_To @fold {
            name @output(out_name: "allegiances")
        }
    }
}'''

args = {
    'character': 'Sansa Stark',
}

In [ ]:
payload = {
    'query': query,
    'args': args,
}

start = time.time()
response = requests.post('http://localhost:5000/graphql', data=json.dumps(payload))
end = time.time()

response_data = json.loads(response.text)
print 'Received response in: {}s'.format(end - start)

print 'Input metadata:'
pprint.pprint(response_data['input_metadata'])

print '\nOutput metadata:'
pprint.pprint(response_data['output_metadata'])

print '\nOutput data:'
pprint.pprint(response_data['output_data'])